<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#practical_plan">Practical Overview</a></li>
        <li><a href="#reading_data">Preparation: importing packages and loading data </a></li>
        <li><a href="#keras_naive">Building a Feedforward Classifier Using Keras</a></li>
        <li><a href="#keras_tuned">Your Task: Using Different Hyperparameters </a></li>
    </ol>
</div>
<br>
<hr>


<h2 id="practical_plan">Practical and Data Overview</h2>

- <b> Aim: </b> using Keras to Build a Simple Neural Network for Classification 
    - Keras is a powerful and easy-to-use free open source Python library for developing and evaluating deep learning models.
    - Keras is now part of Tensorflow
    - Keras wraps the efficient numerical computation libraries performed in Tensorflow and allows you to define and train neural network models in just a few lines of code by specifying the design of the network's layers. 
    - Keras was build by Francois Chollet. Book: https://www.manning.com/books/deep-learning-with-python-second-edition
    - In this tutorial, you will discover how to create your first deep learning neural network model in Python using Keras.
    - We will use the same dataset (prima indians diabetes) in this tutorial. 
- <b>Prerequisites: </b>

    - You need Tensorflow to run this tutorial. You should have installed Tensorflow during a previous practical. However, if you have missed the session, please refer to Jeff Heaton's Youtube blog for instructions: 

        - Windows 10:	
https://www.youtube.com/watch?v=RgO8BBNGB8w&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN

        - MacOS
https://www.youtube.com/watch?v=MpUvdLD932c&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN

    - Alternatively, you can use Google Colab: https://colab.research.google.com/

- <b> Practical Steps: </b> 
    - Load Data.
    - Define Keras Model.
    - Compile Keras Model.
    - Fit Keras Model.
    - Evaluate Keras Model.
    - Make Predictions
    - Repeat the same steps with a more tuned model and examine the performance difference. 


<h2 id="read_data">Loading the data into appropriate variables. </h2>

As in the first practical, we can directly use numpy to load the dataset into two arrays: X (2D array/matrix) and y (1D array). We will need to import the loadtxt library from numpy. 


In [194]:
from numpy import loadtxt
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report


dataset = loadtxt('pima-indians-diabetes.csv', delimiter=',')

# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

X.shape

(768, 8)

Note, the dataset has 9 columns and the range 0:7 will select columns from 0 to 7, stopping before index 8. If this is new to you, then you can learn more about array slicing and ranges in this blog: 

https://machinelearningmastery.com/index-slice-reshape-numpy-arrays-machine-learning-python/

<h2 id="keras_naive">Building a Feed-forward Classifier with Keras</h2>

- We are now ready to define our neural network model.
- Models in Keras are defined as a sequence of layers.

- We create a Sequential model (feed forward network) and add layers one at a time until we are happy with our network architecture.
- The first thing to get right is to ensure the input layer has the right number of input features. This can be specified when creating the first layer with the input_dim argument and setting it to 8 for the 8 input variables.

- How do we know the number of layers and their types?

- This is a very hard question. There are heuristics that we can use and often the best network structure is found through a process of trial and error experimentation. Generally, you need a network large enough to capture the structure of the problem.

- In this example, we will use a fully-connected network structure with three layers.

- Fully connected layers are defined using the Dense class. We can specify the number of neurons or nodes in the layer as the first argument, and specify the activation function using the activation argument.

- We will use the rectified linear unit (Relu) activation function on the first two layers and the Sigmoid function in the output layer.

- It used to be the case that Sigmoid and Tanh activation functions were preferred for all layers. These days, better performance is achieved using the ReLU activation function. We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

- We can piece it all together by adding each layer:
    - The model expects rows of data with 8 variables (the input_dim=8 argument)
    - The first hidden layer has 12 nodes and uses the relu activation function.
    - The second hidden layer has 8 nodes and uses the relu activation function.
    - The output layer has one node and uses the sigmoid activation function.


##### First, we need to import the layers from the keras library

In [195]:
from keras.models import Sequential  ##We are building a sequential (feed forward) model
from keras.layers import Dense       ##Dense layers (we are adding fully connected layers)

In [196]:
# Create the keras model and add layers one by one, indicating the number of neurons and activation function. 

model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))  ##12 = number of neurons, input_dim = number of features
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

<b>Note</b>: the most confusing thing here is that the shape of the input to the model is defined as an argument on the first hidden layer. This means that the line of code that adds the first Dense layer is doing 2 things, defining the input or visible layer and the first hidden layer.

####  Compile Keras Model
Now that the model is defined, we can compile it.

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights and biases to map inputs to outputs in our dataset.

We must specify the loss function (i.e. the error) to use to evaluate a set of weights, the optimizer is used to search through different weights for the network and any optional metrics we would like to collect and report during training.

In this case, we will use cross entropy as the loss argument. This loss is for a binary classification problems and is defined in Keras as <b>binary_crossentropy</b>. You can learn more about choosing loss functions based on your problem here:

https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

We will define the optimizer as the efficient stochastic gradient descent algorithm <b>adam</b>. This is a popular version of gradient descent because it automatically tunes itself and gives good results in a wide range of problems. To learn more about the Adam version of stochastic gradient descent see the post:

https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/
Finally, because it is a classification problem, we will collect and report the classification accuracy, defined via the <b>metrics</b> argument.

The file code to compile network becomes: 

In [197]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



- A good explanation of the difference between the loss and metrics arguments: 

https://stackoverflow.com/questions/48280873/what-is-the-difference-between-loss-function-and-metric-in-keras#:~:text=The%20loss%20function%20is%20used,do%20with%20the%20optimization%20process.

#### Fit Keras Model
We have defined our model and compiled it ready for efficient computation.

Now it is time to execute the model on some data.

We can train or fit our model on our loaded data by calling the fit() function on the model.

Training occurs over epochs and each epoch is split into batches.

- Epoch: One pass through all of the rows in the training dataset.
- Batch: One or more samples considered by the model within an epoch before weights are updated.

One epoch is comprised of one or more batches, based on the chosen batch size and the model is fit for many epochs. For more on the difference between epochs and batches, see the link: 

https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify using the <b>epochs</b> argument. We must also set the number of dataset rows that are considered before the model weights are updated within each epoch, called the batch size and set using the <b>batch_size</b> argument.

For this problem, we will run for a small number of epochs (150) and use a relatively small batch size of 10.

These configurations can be chosen experimentally by trial and error. We want to train the model enough so that it learns a good (or good enough) mapping of rows of input data to the output classification. The model will always have some error, but the amount of error will level out after some point for a given model configuration. This is called model convergence.

In [198]:
# fit the keras model on the dataset
model.fit(x_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
70/70 [==============================] - 1s 542us/step - loss: 32.6440 - accuracy: 0.3557
Epoch 2/150
70/70 [==============================] - 0s 501us/step - loss: 5.2626 - accuracy: 0.5022
Epoch 3/150
70/70 [==============================] - 0s 521us/step - loss: 1.8045 - accuracy: 0.6248
Epoch 4/150
70/70 [==============================] - 0s 527us/step - loss: 1.4432 - accuracy: 0.6510
Epoch 5/150
70/70 [==============================] - 0s 569us/step - loss: 1.4795 - accuracy: 0.6224
Epoch 6/150
70/70 [==============================] - 0s 466us/step - loss: 1.2922 - accuracy: 0.6457
Epoch 7/150
70/70 [==============================] - 0s 475us/step - loss: 1.2500 - accuracy: 0.6289
Epoch 8/150
70/70 [==============================] - 0s 475us/step - loss: 1.2032 - accuracy: 0.6038
Epoch 9/150
70/70 [==============================] - 0s 471us/step - loss: 0.9853 - accuracy: 0.6322
Epoch 10/150
70/70 [==============================] - 0s 545us/step - loss: 0.8562 - accur

####  Evaluate Keras Model
We have trained our neural network on the training dataset and we can evaluate the performance of the network on the test dataset. Let's first use the model to predict the actual classes for the test set

Making predictions is as easy as calling the predict() function on the model. We are using a sigmoid activation function on the output layer, so the predictions will be a probability in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

ou can visualise your predictions using any of the sklearn capabilities. Example below shows the classification report.

For example: 


In [199]:
# make probability predictions with the model
predictions = model.predict(x_test)
# round predictions 
rounded = [round(x[0]) for x in predictions]
print(classification_report(y_test, rounded))

              precision    recall  f1-score   support

         0.0       0.75      0.96      0.84        51
         1.0       0.83      0.38      0.53        26

    accuracy                           0.77        77
   macro avg       0.79      0.67      0.69        77
weighted avg       0.78      0.77      0.74        77



<h2 id="keras_tuned">Changing the Model's Hyperparameters</h2>

 - The NN above shows a very bad performance with respect to the positive class. 
 - Can we do better? let's try differnt parameters: 
     - batch_size
     - epochs
     - metrics (try: metrics=tf.keras.metrics.Precision())


In [200]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=150, batch_size=10)



Epoch 1/150
70/70 [==============================] - 0s 531us/step - loss: 0.4804 - accuracy: 0.7679
Epoch 2/150
70/70 [==============================] - 0s 472us/step - loss: 0.5631 - accuracy: 0.7097
Epoch 3/150
70/70 [==============================] - 0s 460us/step - loss: 0.5035 - accuracy: 0.7452
Epoch 4/150
70/70 [==============================] - 0s 460us/step - loss: 0.5040 - accuracy: 0.7599
Epoch 5/150
70/70 [==============================] - 0s 478us/step - loss: 0.5188 - accuracy: 0.7308
Epoch 6/150
70/70 [==============================] - 0s 464us/step - loss: 0.5273 - accuracy: 0.7230
Epoch 7/150
70/70 [==============================] - 0s 467us/step - loss: 0.4766 - accuracy: 0.7573
Epoch 8/150
70/70 [==============================] - 0s 529us/step - loss: 0.6106 - accuracy: 0.6929
Epoch 9/150
70/70 [==============================] - 0s 465us/step - loss: 0.5373 - accuracy: 0.7100
Epoch 10/150
70/70 [==============================] - 0s 472us/step - loss: 0.5077 - accura

In [201]:
# make probability predictions with the model
predictions = model.predict(x_test)
# round predictions 
rounded = [round(x[0]) for x in predictions]
print(classification_report(y_test, rounded))

              precision    recall  f1-score   support

         0.0       0.80      0.76      0.78        51
         1.0       0.57      0.62      0.59        26

    accuracy                           0.71        77
   macro avg       0.68      0.69      0.69        77
weighted avg       0.72      0.71      0.72        77



In [202]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=tf.keras.metrics.Precision())
model.fit(x_train, y_train, epochs=200, batch_size=10)
# make probability predictions with the model



Epoch 1/200
70/70 [==============================] - 0s 527us/step - loss: 0.4549 - precision_11: 0.6985
Epoch 2/200
70/70 [==============================] - 0s 462us/step - loss: 0.4708 - precision_11: 0.7103
Epoch 3/200
70/70 [==============================] - 0s 457us/step - loss: 0.4764 - precision_11: 0.6665
Epoch 4/200
70/70 [==============================] - 0s 459us/step - loss: 0.4689 - precision_11: 0.7043
Epoch 5/200
70/70 [==============================] - 0s 462us/step - loss: 0.4843 - precision_11: 0.7516
Epoch 6/200
70/70 [==============================] - 0s 478us/step - loss: 0.5159 - precision_11: 0.7347
Epoch 7/200
70/70 [==============================] - 0s 457us/step - loss: 0.4569 - precision_11: 0.7450
Epoch 8/200
70/70 [==============================] - 0s 469us/step - loss: 0.4783 - precision_11: 0.6736
Epoch 9/200
70/70 [==============================] - 0s 493us/step - loss: 0.4615 - precision_11: 0.6968
Epoch 10/200
70/70 [==============================] - 0

70/70 [==============================] - 0s 1ms/step - loss: 0.4166 - precision_11: 0.7469
Epoch 155/200
70/70 [==============================] - 0s 543us/step - loss: 0.4408 - precision_11: 0.7427
Epoch 156/200
70/70 [==============================] - 0s 500us/step - loss: 0.4588 - precision_11: 0.7260
Epoch 157/200
70/70 [==============================] - 0s 465us/step - loss: 0.4249 - precision_11: 0.7884
Epoch 158/200
70/70 [==============================] - 0s 457us/step - loss: 0.4153 - precision_11: 0.7628
Epoch 159/200
70/70 [==============================] - 0s 469us/step - loss: 0.4546 - precision_11: 0.7183
Epoch 160/200
70/70 [==============================] - 0s 457us/step - loss: 0.5183 - precision_11: 0.6845
Epoch 161/200
70/70 [==============================] - 0s 465us/step - loss: 0.4430 - precision_11: 0.7507
Epoch 162/200
70/70 [==============================] - 0s 458us/step - loss: 0.4345 - precision_11: 0.7282
Epoch 163/200
70/70 [==============================] 

In [203]:
predictions = model.predict(x_test)
# round predictions 
rounded = [round(x[0]) for x in predictions]
print(classification_report(y_test, rounded))

              precision    recall  f1-score   support

         0.0       0.81      0.75      0.78        51
         1.0       0.57      0.65      0.61        26

    accuracy                           0.71        77
   macro avg       0.69      0.70      0.69        77
weighted avg       0.73      0.71      0.72        77



In [204]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mae'])
model.fit(x_train, y_train, epochs=50, batch_size=10)



Epoch 1/50
70/70 [==============================] - 0s 499us/step - loss: 0.4483 - mae: 0.2941
Epoch 2/50
70/70 [==============================] - 0s 444us/step - loss: 0.4577 - mae: 0.2947
Epoch 3/50
70/70 [==============================] - 0s 436us/step - loss: 0.4249 - mae: 0.2809
Epoch 4/50
70/70 [==============================] - 0s 435us/step - loss: 0.4593 - mae: 0.2942
Epoch 5/50
70/70 [==============================] - 0s 434us/step - loss: 0.4451 - mae: 0.2700
Epoch 6/50
70/70 [==============================] - 0s 584us/step - loss: 0.4890 - mae: 0.3079
Epoch 7/50
70/70 [==============================] - 0s 473us/step - loss: 0.4527 - mae: 0.2957
Epoch 8/50
70/70 [==============================] - 0s 437us/step - loss: 0.4195 - mae: 0.2832
Epoch 9/50
70/70 [==============================] - 0s 462us/step - loss: 0.4226 - mae: 0.2797
Epoch 10/50
70/70 [==============================] - 0s 443us/step - loss: 0.4517 - mae: 0.2933
Epoch 11/50
70/70 [==============================

In [205]:
predictions = model.predict(x_test)
# round predictions 
rounded = [round(x[0]) for x in predictions]
print(classification_report(y_test, rounded))

              precision    recall  f1-score   support

         0.0       0.79      0.80      0.80        51
         1.0       0.60      0.58      0.59        26

    accuracy                           0.73        77
   macro avg       0.69      0.69      0.69        77
weighted avg       0.72      0.73      0.73        77



In [206]:
import tensorflow as tf
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=tf.keras.metrics.Precision())
model.fit(x_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
70/70 [==============================] - 0s 512us/step - loss: 0.4319 - precision_12: 0.7651
Epoch 2/150
70/70 [==============================] - 0s 463us/step - loss: 0.4271 - precision_12: 0.7357
Epoch 3/150
70/70 [==============================] - 0s 461us/step - loss: 0.4376 - precision_12: 0.7603
Epoch 4/150
70/70 [==============================] - 0s 496us/step - loss: 0.3990 - precision_12: 0.7553
Epoch 5/150
70/70 [==============================] - 0s 459us/step - loss: 0.4419 - precision_12: 0.7538
Epoch 6/150
70/70 [==============================] - 0s 465us/step - loss: 0.4070 - precision_12: 0.7534
Epoch 7/150
70/70 [==============================] - 0s 490us/step - loss: 0.4408 - precision_12: 0.7604
Epoch 8/150
70/70 [==============================] - 0s 468us/step - loss: 0.4391 - precision_12: 0.7079
Epoch 9/150
70/70 [==============================] - 0s 486us/step - loss: 0.4227 - precision_12: 0.7008
Epoch 10/150
70/70 [==============================] - 0

In [207]:
# make probability predictions with the model
predictions = model.predict(x_test)
# round predictions 
rounded = [round(x[0]) for x in predictions]
print(classification_report(y_test, rounded))

              precision    recall  f1-score   support

         0.0       0.81      0.75      0.78        51
         1.0       0.57      0.65      0.61        26

    accuracy                           0.71        77
   macro avg       0.69      0.70      0.69        77
weighted avg       0.73      0.71      0.72        77



In [208]:
## the model still performs badly, especially w.r.t. the positive class. There's a big class imbalance, see here: 
import numpy as np

values, counts = np.unique(y, return_counts=True)
print(counts)
##there is double the number of samples in the negative class. 
## NNs are very data hungry. they would require much more data about the positive class to make good predictions!
## We will learn how to deal with class imbalance next week! :) 

[500 268]
